<a href="https://colab.research.google.com/github/donbcolab/AIE3/blob/main/paligemma_tumor_finetune_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q -U git+https://github.com/huggingface/transformers.git bitsandbytes datasets accelerate peft hf_transfer torch torchvision

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19

In [ ]:
import torch
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig
from transformers import AutoProcessor
from datasets import load_dataset, DatasetDict, Image
from google.colab import userdata
import os

# Constants
base_model_name = "google/paligemma-3b-pt-224"
adapter_version = "paligemma-tumor-detection-ft"
adapter_model_name = f"dwb2023/{adapter_version}"
max_seq_length = 512
output_dir = adapter_version

# HF Token
HF_TOKEN = userdata.get('HF_TOKEN')
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Dataset
dataset = load_dataset("dwb2023/brain-tumor-image-dataset-semantic-segmentation")

# Preprocess Dataset
processor = AutoProcessor.from_pretrained(base_model_name)
tumor_dataset = dataset

# Split Dataset
train_test_split = tumor_dataset['train'].train_test_split(test_size=0.2)
test_val_split = train_test_split['test'].train_test_split(test_size=0.5)

In [ ]:
# Create Dataset Dictionary
tumor_ds_dict = DatasetDict({
    'train': train_test_split['train'],
    'validation': test_val_split['train'],
    'test': test_val_split['test']
})

In [ ]:
# Cast image column AFTER splitting the dataset
ds_train = tumor_ds_dict["train"].cast_column("image", Image(decode=True))
ds_eval = tumor_ds_dict["validation"].cast_column("image", Image(decode=True))

In [ ]:
# Collate Function (optimized)
device = "cuda"

def collate_fn(examples):
    texts = ["Detect the location of the brain tumor using a bounding box: " for _ in range(len(examples))]
    labels = [str(example['bbox']) for example in examples]  # Convert bbox to string

    # Image conversion on CPU
    images = [example["image"].convert("RGB") for example in examples]

    # Image conversion on CPU
    images = [example["image"].convert("RGB") for example in examples]

    tokens = processor(text=texts, images=images, suffix=labels,
                       return_tensors="pt", padding="longest", max_length=max_seq_length)

    # Convert to bfloat16 before moving to GPU
    tokens = tokens.to(torch.bfloat16).to(device)
    return tokens

In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_type=torch.bfloat16
)

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)
model = PaliGemmaForConditionalGeneration.from_pretrained(base_model_name, quantization_config=bnb_config, device_map={"":0})
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Determine the dataset size
num_train_samples = len(ds_train)
print(f"Number of training samples: {num_train_samples}")
num_epochs = 100
batch_size = 2
gradient_accumulation_steps = 8

# Calculate total training steps
steps_per_epoch = num_train_samples // (batch_size * gradient_accumulation_steps)
total_training_steps = steps_per_epoch * num_epochs

# Set warmup steps to 10% of total training steps
warmup_steps = int(0.1 * total_training_steps)


args=TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_epochs,
    remove_unused_columns=False,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    warmup_steps=warmup_steps,
    learning_rate=2e-5,
    weight_decay=1e-6,
    adam_beta2=0.999,
    logging_steps=50,
    optim="adamw_hf",
    save_strategy="epoch",
    push_to_hub=True,
    save_total_limit=1,
    bf16=True,
    report_to=["tensorboard"],
    dataloader_pin_memory=False,
    load_best_model_at_end=True,  # Required for EarlyStoppingCallback
    evaluation_strategy="epoch",  # Set to epoch for periodic evaluation
)

In [ ]:
from transformers import Trainer, EarlyStoppingCallback

# Define EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
  early_stopping_patience=3,
  early_stopping_threshold=0.01,
)

# Define Trainer with EarlyStoppingCallback
trainer = Trainer(
  model=model,
  train_dataset=ds_train,
  eval_dataset=ds_eval,
  data_collator=collate_fn,
  args=args,
  callbacks=[early_stopping]
)

In [ ]:
# Train model
trainer.train()

In [ ]:
# Evaluate Model
metrics = trainer.evaluate()
print(metrics)